In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
master = pd.read_csv("data/Master.csv")
batting = pd.read_csv("data/Batting.csv")
salaries = pd.read_csv("data/Salaries.csv")
positions = pd.read_csv("data/Appearances.csv")
pitching = pd.read_csv("data/Pitching.csv")

In [4]:
part_master = master[["playerID", "nameFirst", "nameLast"]]
part_batting = batting[["playerID", "yearID", "teamID", "lgID", "G", "AB", "H", "BB", "HBP", "SF"]]
part_salaries = salaries[["playerID", "yearID", "teamID", "salary"]]
part_positions = positions[["playerID", "yearID", "teamID", "GS", "G_p", "G_c", "G_1b", "G_2b", "G_3b", "G_ss", "G_lf", "G_cf", "G_rf", "G_of", "G_dh"]]
part_pitching = pitching[["playerID", "yearID", "teamID", "W", "L", "GS", "ERA"]]

In [5]:
batting_2004 = part_batting[part_batting["yearID"] == 2004]
salaries_2004 = part_salaries[part_salaries["yearID"] == 2004]
positions_2004 = part_positions[part_positions["yearID"] == 2004]
pitching_2004 = part_pitching[part_pitching["yearID"] == 2004]

In [69]:
pitching_2004.sort("W", ascending=False).head()

,playerID,yearID,teamID,W,L,GS,ERA
36106,schilcu01,2004,BOS,21,6,32,3.26
36104,santajo01,2004,MIN,20,6,34,2.61
36019,oswalro01,2004,HOU,20,10,35,3.49
36107,schmija01,2004,SFN,18,7,32,3.20
35676,clemero02,2004,HOU,18,4,33,2.98


In [6]:
battpitch_2004 = pd.merge(batting_2004, pitching_2004, how="outer", on=("playerID", "teamID"))

In [7]:
battpitchpos_2004 = pd.merge(battpitch_2004, positions_2004, how="outer", on=("playerID", "teamID"))

In [8]:
battpitchpossal_2004 = pd.merge(battpitchpos_2004, salaries_2004, how="outer", on=("playerID", "teamID"))

In [10]:
mlb_2004 = pd.merge(battpitchpossal_2004, part_master, how="inner", on="playerID")

In [14]:
mlb_2004["OBP"] = (mlb_2004.H + mlb_2004.BB + mlb_2004.HBP) / (mlb_2004.AB + mlb_2004.BB + mlb_2004.HBP + mlb_2004.SF)

In [16]:
OBP_2004 = mlb_2004.OBP

In [18]:
avg_obp_2004 = OBP_2004.mean()

In [21]:
stdev_obp_2004 = OBP_2004.std()

In [24]:
mlb_2004["OBP_std"] = (mlb_2004.OBP - avg_obp_2004) / stdev_obp_2004

In [45]:
avg_sal_2004 = mlb_2004.salary.mean()
stdev_sal_2004 = mlb_2004.salary.std()

In [70]:
avg_sal_2004

2489998.6775000002

In [46]:
mlb_2004["salary_std"] = (mlb_2004.salary - avg_sal_2004) / stdev_sal_2004

In [47]:
mlb_2004["OBP_Sal"] = mlb_2004.OBP_std - mlb_2004.salary_std

In [102]:
mlb_2004.sort("OBP_Sal", ascending=False).head(10)

,playerID,yearID_x,teamID,lgID,G,AB,H,BB,HBP,SF,...,G_of,G_dh,yearID_y,salary,nameFirst,nameLast,OBP,OBP_std,salary_std,OBP_Sal
258,cottsne01,2004,CHA,AL,56,1,1,0,0,0,...,0,0,2004,301000,Neal,Cotts,1.000000,5.033774,-0.619846,5.653620
146,broweji01,2004,SFN,NL,89,2,1,1,0,0,...,0,0,2004,662500,Jim,Brower,0.666667,2.806724,-0.517482,3.324206
352,eischjo01,2004,MON,NL,22,3,2,0,0,0,...,0,0,2004,1300000,Joey,Eischen,0.666667,2.806724,-0.336965,3.143689
1207,torcato01,2004,SFN,NL,13,9,5,1,1,1,...,0,0,2004,302000,Tony,Torcato,0.583333,2.249961,-0.619563,2.869524
97,bentzch01,2004,MON,NL,36,2,1,0,0,0,...,0,0,2004,300000,Chad,Bentz,0.500000,1.693199,-0.620129,2.313328
675,knottga01,2004,DET,AL,36,3,1,1,0,0,...,0,0,2004,316000,Gary,Knotts,0.500000,1.693199,-0.615599,2.308797
34,ankieri01,2004,SLN,NL,5,1,0,1,0,0,...,0,0,2004,320000,Rick,Ankiel,0.500000,1.693199,-0.614466,2.307665
922,olmedra01,2004,CIN,NL,8,1,0,1,0,0,...,0,0,2004,322000,Ray,Olmedo,0.500000,1.693199,-0.613900,2.307098
457,gipsoch01,2004,TBA,AL,5,4,2,0,0,0,...,2,0,2004,350000,Charles,Gipson,0.500000,1.693199,-0.605971,2.299170
1131,simonja01,2004,SLN,NL,15,2,1,0,0,0,...,1,0,2004,350000,Jason,Simontacchi,0.500000,1.693199,-0.605971,2.299170


At this point, I have a nice dataframe called mlb_2004 that has batting, pitching, appearance, and salary stats from the 2004 MLB season. I've also added in: OBP (on base percentage), OBP_std (how many standard deviations above or below the mean OBP that player is), salary_std (how many standard deviations above or below the mean salary that player is), OBP_Sal (OBP_std minus salary_std).

This last number can help compare players based on their OBP relative to the average along with their salary relative to the average.

In [50]:
mlb_2004_pitchers = mlb_2004[mlb_2004["G_p"] > 0]

In [77]:
mlb_2004_pitchers = mlb_2004_pitchers[mlb_2004_pitchers.OBP.notnull()]

In [105]:
mlb_2014_pitchers[mlb_2014_pitchers["lgID"] == "NL"].sort("W", ascending=False).head(20)

,playerID,yearID_x,teamID,lgID,G,AB,H,BB,HBP,SF,...,G_of,G_dh,yearID_y,salary,nameFirst,nameLast,OBP,OBP_std,salary_std,OBP_Sal
933,oswalro01,2004,HOU,NL,36,71,10,3,1,1,...,0,0,2004,3250000,Roy,Oswalt,0.184211,-0.416638,0.215205,-0.631843
949,pavanca01,2004,FLO,NL,31,68,13,1,1,0,...,0,0,2004,3800000,Carl,Pavano,0.214286,-0.215701,0.370945,-0.586647
233,clemero02,2004,HOU,NL,33,72,12,3,0,0,...,0,0,2004,5000000,Roger,Clemens,0.200000,-0.311146,0.710742,-1.021889
1109,schmija01,2004,SFN,NL,32,66,9,4,1,0,...,0,0,2004,7937500,Jason,Schmidt,0.197183,-0.329967,1.542537,-1.872504
1172,suppaje01,2004,SLN,NL,31,57,4,1,0,0,...,0,0,2004,1000000,Jeff,Suppan,0.086207,-1.071415,-0.421914,-0.649501
1339,zambrca01,2004,CHN,NL,31,70,16,3,0,1,...,0,0,2004,450000,Carlos,Zambrano,0.256757,0.068054,-0.577655,0.645709
768,maddugr01,2004,CHN,NL,34,69,11,1,0,0,...,0,0,2004,6000000,Greg,Maddux,0.171429,-0.502036,0.993907,-1.495943
639,johnsra05,2004,ARI,NL,35,80,10,4,0,0,...,0,0,2004,16000000,Randy,Johnson,0.166667,-0.533851,3.825548,-4.359400
700,lawrebr02,2004,SDN,NL,36,62,6,3,0,0,...,0,0,2004,925000,Brian,Lawrence,0.138462,-0.722294,-0.443152,-0.279143
780,marquja01,2004,SLN,NL,35,72,21,1,0,1,...,0,0,2004,525000,Jason,Marquis,0.297297,0.338911,-0.556417,0.895329


Johan Santana from Minnesota is a good candidate for starting pitcher. His record in 2004 was 20-6, with a 2.61 ERA, so he's an excellent pitcher. But he's also a decent hitter. His OBP that year was .375, which is not quite 1 full standard deviation above the mean (0.858 to be exact), and his salary was just below the mean (1.6 million to the avg of 2.5 million). So his OBP_Sal is 1.11, which is much better than any of the other pitchers around him based on total wins. Unfortunately, he's an AL pitcher, so he only had 8 ABs, which means his OBP might be an anomoly. So we'll restrict this to NL pitchers, for the sake of accuracy on OBP. With that restriction, the best pitcher is either Carlos Zambrano of the Cubs (16-8, 2.75 ERA, .257 OBP, Salary 450,000, 70 ABs, .646 OBP_Sal) or Jason Marquis of the Cardinals (15-7, 3.71 ERA, .297 OBP, Salary 525,000, 72 ABs, .895 OBP_Sal). Since this homework is supposed to focus on OBP, and since I'm a Cardinals fan, I'll go with Marquis on this one.

In [107]:
mlb_2004_catch = mlb_2004[mlb_2004["G_c"] > 0].sort("OBP_Sal", ascending=False)

In [109]:
mlb_2004_catch = mlb_2004_catch[["playerID", "teamID", "lgID", "G", "AB", "H", "BB", "G_c", "salary", "nameFirst", "nameLast", "OBP", "OBP_std", "salary_std", "OBP_Sal"]]

In [111]:
mlb_2004_catch.head(10)

,playerID,teamID,lgID,G,AB,H,BB,G_c,salary,nameFirst,nameLast,OBP,OBP_std,salary_std,OBP_Sal
60,bardjo01,CLE,AL,7,19,8,3,7,316700,Josh,Bard,0.478261,1.547956,-0.615400,2.163357
372,estrajo01,ATL,NL,134,462,145,39,133,312500,Johnny,Estrada,0.377907,0.877477,-0.616590,1.494066
1166,stinnke01,KCA,AL,20,59,18,5,20,500000,Kelly,Stinnett,0.378788,0.883362,-0.563496,1.446859
804,mauerjo01,MIN,AL,35,107,33,11,32,300000,Joe,Mauer,0.368852,0.816982,-0.620129,1.437111
793,martivi01,CLE,AL,141,520,147,60,132,304500,Victor,Martinez,0.358714,0.749246,-0.618855,1.368101
849,mirabdo01,BOS,AL,59,160,45,19,53,825000,Doug,Mirabelli,0.368132,0.812168,-0.471468,1.283636
617,ingebr01,DET,AL,131,408,117,32,39,340000,Brandon,Inge,0.340000,0.624215,-0.608803,1.233017
998,prattto02,PHI,NL,45,128,33,18,43,875000,Todd,Pratt,0.351351,0.700055,-0.457310,1.157365
1110,schnebr01,MON,NL,135,436,112,42,133,350000,Brian,Schneider,0.325052,0.524343,-0.605971,1.130314
915,ojedami01,SDN,NL,62,156,40,15,50,305000,Miguel,Ojeda,0.321839,0.502879,-0.618713,1.121592


At catcher, I'm going to take Johnny Estrada of the Braves, with an OBP of .378, a salary of 312,500, and an OBP_Sal of 1.494 over 462 ABs. (For the sake of space, all of the following position player dataframes will only be displayed to 10 rows instead of 20.)